# Ciminion DRL Gröbner Basis
DRL Gröbner basis computation for an arbitrary Ciminion instance.

In [1]:
load("Ciminion.sage")
load("Ciminion_polynomial_model.sage")

## Ciminion Instance

In [2]:
ciminion = Ciminion(info_level=1)

Ciminion parameters
Field: Finite Field of size 170141183460469231731687303715884105773
Rounds_C: 90
Rounds_E: 14
Constants_C: [[92368723631891307050466755452684648324, 97456644815185509926671064466649619100, 142812243190817652640517864183798444015, 122487605141255997594482114390180971740], [29907925740593477655334356327506826403, 113602593930577118106253270075487554118, 102042668346553826930951731053937798348, 123493006927229273525417566013628340445], [110193465846200426034312352314782456091, 156306017716408743317883092069761922912, 34877654923465784290510695525142525238, 128673103331879550282396569622012065416], [154627597893828623788010640056548809974, 162948277714965990676432475517805924457, 47875903142326301236265400305063133260, 5692363000814608683547541765470082306], [9803361022917620011819704633999901027, 3416699216689659911668308657094037949, 145183371900868523888943781742979142503, 113109977626588497377113692692584229198], [90642723834291886538400270842464575182, 143945760580

## Ciminion Polynomial model

In [3]:
polys = generate_Ciminion_polynomials(ciminion=ciminion)

P = polys[0].parent()

Plaintext: [28080838834753758700552709093715238361, 142060849449112374167608085159827058300]
Key: [20009386238576363272322356893582955622, 5919821820550523439359915232188584243]
Nonce: 113992257709080166183684367649876011999
Ciphertext: [106120835172215164462566696907254767877, 47743660755865453046073681128095141036]
Term order: degrevlex


## DRL Gröbner Basis computation
First we change to a different DRL term order.

In [4]:
f = efficient_Ciminion_termorder(ciminion, P)

Next we invert the matrices in the round functions, extract the linear polynomials in each round, and perform Gaussian elimination on the linear polynomials.

In [5]:
polys = [f(poly) for poly in transform_Ciminion_polynomial_system(ciminion, polys)]

lin_polys = Sequence([poly for poly in polys if poly.degree() == 1])
M, v = lin_polys.coefficients_monomials()

M = M.echelon_form()

lin_polys = ideal((M * v).list())

Now we reduce the non-linear polynomials with respect to the linear polynomials, i.e. we eliminate variables in the non-linear polynomials.
We call the resulting polynomial system the downsized Gröbner basis.

In [6]:
gb_downsized = [poly.reduce(lin_polys) for poly in polys if poly.degree() > 1]
lms = [poly.lm() for poly in gb_downsized]

In [7]:
for mon in lms:
    print(mon)

x_C_3__2^2
x_C_3__3^2
x_C_3__4^2
x_C_3__5^2
x_C_3__6^2
x_C_3__7^2
x_C_3__8^2
x_C_3__9^2
x_C_3__10^2
x_C_3__11^2
x_C_3__12^2
x_C_3__13^2
x_C_3__14^2
x_C_3__15^2
x_C_3__16^2
x_C_3__17^2
x_C_3__18^2
x_C_3__19^2
x_C_3__20^2
x_C_3__21^2
x_C_3__22^2
x_C_3__23^2
x_C_3__24^2
x_C_3__25^2
x_C_3__26^2
x_C_3__27^2
x_C_3__28^2
x_C_3__29^2
x_C_3__30^2
x_C_3__31^2
x_C_3__32^2
x_C_3__33^2
x_C_3__34^2
x_C_3__35^2
x_C_3__36^2
x_C_3__37^2
x_C_3__38^2
x_C_3__39^2
x_C_3__40^2
x_C_3__41^2
x_C_3__42^2
x_C_3__43^2
x_C_3__44^2
x_C_3__45^2
x_C_3__46^2
x_C_3__47^2
x_C_3__48^2
x_C_3__49^2
x_C_3__50^2
x_C_3__51^2
x_C_3__52^2
x_C_3__53^2
x_C_3__54^2
x_C_3__55^2
x_C_3__56^2
x_C_3__57^2
x_C_3__58^2
x_C_3__59^2
x_C_3__60^2
x_C_3__61^2
x_C_3__62^2
x_C_3__63^2
x_C_3__64^2
x_C_3__65^2
x_C_3__66^2
x_C_3__67^2
x_C_3__68^2
x_C_3__69^2
x_C_3__70^2
x_C_3__71^2
x_C_3__72^2
x_C_3__73^2
x_C_3__74^2
x_C_3__75^2
x_C_3__76^2
x_C_3__77^2
x_C_3__78^2
x_C_3__79^2
x_C_3__80^2
x_C_3__81^2
x_C_3__82^2
x_C_3__83^2
x_C_3__84^2
x_C_3__85^2


In [8]:
gcds = [gcd(comb[0], comb[1]) for comb in Combinations(lms, 2)]
list(set(gcds))

[1]

Since all leading monomials are pairwise coprime we have indeed constructed a DRL Gröbner basis.

Finally, let us verify that the downsized Gröbner basis is indeed zero_dimensional by extracting the variables present in the downsized system.

In [9]:
P = gb_downsized[0].parent()
variables = P.gens()
eliminated_variables = [poly.lm() for poly in lin_polys.gens()]
variables_downsized = [var for var in variables if not var in eliminated_variables]

In [10]:
ideal([var**2 for var in variables_downsized]) == ideal(lms)

True

In [11]:
M, v = Sequence(gb_downsized).coefficients_monomials()
M = M.echelon_form()

gb_downsized_red = list((vector(M * v)))

In [12]:
sep = 100 * "-"

for poly in gb_downsized_red:
    print(poly)
    print(sep)

x_C_3__2^2 + 155739314636472885460618739657404860414*x_C_3__2*x_C_3__3 + 68218853678747695023323393984863351361*x_C_3__2*x_C_3__4 + 29964994529287399176351440987907081577*x_C_3__2*x_C_3__5 + 90841156421844806088392901343601072548*x_C_3__2*x_C_3__6 + 54014834830825438349814634187093179863*x_C_3__2*x_C_3__7 + 28183278613515937436640870019633222805*x_C_3__2*x_C_3__8 + 77867740810986851964093119079428396785*x_C_3__2*x_C_3__9 + 85122993062861890063615703679763173379*x_C_3__2*x_C_3__10 + 116700530754119853837981332215557018903*x_C_3__2*x_C_3__11 + 80322217255720153201123538451862255941*x_C_3__2*x_C_3__12 + 3392623000249159273212310600295090048*x_C_3__2*x_C_3__13 + 155628841903803963581869078288848776273*x_C_3__2*x_C_3__14 + 66718619728056537381382375116407916396*x_C_3__2*x_C_3__15 + 143069597755572161337858807824992643219*x_C_3__2*x_C_3__16 + 79825264662140544772622708407825161137*x_C_3__2*x_C_3__17 + 102324846268342236175328365403757987293*x_C_3__2*x_C_3__18 + 370947112088848451632519481363

In [13]:
set(flatten([poly.variables() for poly in gb_downsized_red])) == set(flatten([poly.lm().variables() for poly in gb_downsized_red]))

True